In [1]:
%pip install --upgrade --user tensorflow==2.3.1

Note: you may need to restart the kernel to use updated packages.


In [2]:
import absl
import os
import tempfile
import time

import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
import tfx

from pprint import pprint
from tensorflow_metadata.proto.v0 import schema_pb2, statistics_pb2, anomalies_pb2
from tensorflow_transform.tf_metadata import schema_utils
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import InfraValidator
from tfx.components import Pusher
from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components import Tuner
from tfx.dsl.components.base import executor_spec
from tfx.components.common_nodes.importer_node import ImporterNode
from tfx.components.trainer import executor as trainer_executor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import evaluator_pb2
from tfx.proto import example_gen_pb2
from tfx.proto import infra_validator_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.proto.evaluator_pb2 import SingleSlicingSpec

from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import HyperParameters
from tfx.types.standard_artifacts import ModelBlessing
from tfx.types.standard_artifacts import InfraBlessing

2023-04-20 04:57:39.992654: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-04-20 04:57:39.992708: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
print("Tensorflow Version:", tf.__version__)
print("TFX Version:", tfx.__version__)
print("TFDV Version:", tfdv.__version__)
print("TFMA Version:", tfma.VERSION_STRING)

Tensorflow Version: 2.3.1
TFX Version: 0.25.0
TFDV Version: 0.25.0
TFMA Version: 0.25.0


In [4]:
os.environ['PATH'] += os.pathsep + '/home/jupyter/.local/bin'

In [5]:
ARTIFACT_STORE = os.path.join(os.sep, 'home', 'jupyter', 'artifact-store')
SERVING_MODEL_DIR=os.path.join(os.sep, 'home', 'jupyter', 'serving_model')
DATA_ROOT = 'gs://tfx_pipeline_bucket'

In [6]:
PIPELINE_NAME = 'tfx-churn-classifier'
PIPELINE_ROOT = os.path.join(ARTIFACT_STORE, PIPELINE_NAME, time.strftime("%Y%m%d_%H%M%S"))
os.makedirs(PIPELINE_ROOT, exist_ok=True)

context = InteractiveContext(
    pipeline_name=PIPELINE_NAME,
    pipeline_root=PIPELINE_ROOT,
    metadata_connection_config=None)

In [104]:
output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[        
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=4),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1)
    ]))

example_gen = tfx.components.CsvExampleGen(
    input_base=DATA_ROOT,
    output_config=output_config)

In [105]:
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 30
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "/home/jupyter/artifact-store/tfx-churn-classifier/20230420_045852/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:561600,xor_checksum:1681914056,sum_checksum:1681914056"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            string_value: "0"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        create_time_since_epoch: 1681966761396
        last_update_time_since_epoch: 1681975201817
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
        ))

In [106]:
examples_uri = example_gen.outputs['examples'].get()[0].uri
tfrecord_filenames = [os.path.join(examples_uri, 'train', name)
                      for name in os.listdir(os.path.join(examples_uri, 'train'))]
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
for tfrecord in dataset.take(2):
  example = tf.train.Example()
  example.ParseFromString(tfrecord.numpy())
  for name, feature in example.features.feature.items():
    if feature.HasField('bytes_list'):
        value = feature.bytes_list.value
    if feature.HasField('float_list'):
        value = feature.float_list.value
    if feature.HasField('int64_list'):
        value = feature.int64_list.value
    print('{}: {}'.format(name, value))
  print('******')

gender: [b'Female']
churn: [1]
tenure: [2]
estimated_salary: [101348.8828125]
active_member: [1]
age: [42]
credit_card: [1]
credit_score: [619]
products_number: [1]
customer_id: [15634602]
balance: [0.0]
country: [b'France']
******
credit_card: [1]
gender: [b'Female']
balance: [159660.796875]
country: [b'France']
customer_id: [15619304]
age: [42]
products_number: [3]
active_member: [0]
tenure: [8]
estimated_salary: [113931.5703125]
credit_score: [502]
churn: [1]
******


In [107]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])

In [108]:
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 31
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 7
        uri: "/home/jupyter/artifact-store/tfx-churn-classifier/20230420_045852/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        create_time_since_epoch: 1681966898269
        last_update_time_since_epoch: 1681967451198
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
        ))

In [109]:
context.show(statistics_gen.outputs['statistics'])

In [110]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)

In [111]:
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 32
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 9
        uri: "/home/jupyter/artifact-store/tfx-churn-classifier/20230420_045852/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        create_time_since_epoch: 1681966951896
        last_update_time_since_epoch: 1681966951914
        , artifact_type: id: 9
        name: "Schema"
        )]
        ))

In [112]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'country',STRING,required,single,'country'
'gender',STRING,required,single,'gender'
'active_member',INT,required,single,-
'age',INT,required,single,-
'balance',FLOAT,required,single,-
'churn',INT,required,single,-
'credit_card',INT,required,single,-
'credit_score',INT,required,single,-
'customer_id',INT,required,single,-


/home/jupyter/.local/lib/python3.7/site-packages/tensorflow_data_validation/utils/display_util.py:151: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'country',"'France', 'Germany', 'Spain'"
'gender',"'Female', 'Male'"


In [113]:
schema_proto_path = '{}/{}'.format(schema_gen.outputs['schema'].get()[0].uri, 'schema.pbtxt')
schema = tfdv.load_schema_text(schema_proto_path)

In [114]:
tfdv.set_domain(schema, 'churn', schema_pb2.IntDomain(name='churn', min=0, max=1, is_categorical=True))
tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'country',STRING,required,single,'country'
'gender',STRING,required,single,'gender'
'active_member',INT,required,single,-
'age',INT,required,single,-
'balance',FLOAT,required,single,-
'churn',INT,required,single,"[0,1]"
'credit_card',INT,required,single,-
'credit_score',INT,required,single,-
'customer_id',INT,required,single,-


/home/jupyter/.local/lib/python3.7/site-packages/tensorflow_data_validation/utils/display_util.py:151: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'country',"'France', 'Germany', 'Spain'"
'gender',"'Female', 'Male'"


In [115]:
schema_dir = os.path.join(ARTIFACT_STORE, 'schema')
tf.io.gfile.makedirs(schema_dir)
schema_file = os.path.join(schema_dir, 'schema.pbtxt')

tfdv.write_schema_text(schema, schema_file)

!cat {schema_file}

feature {
  name: "country"
  value_count {
    min: 1
    max: 1
  }
  type: BYTES
  domain: "country"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "gender"
  value_count {
    min: 1
    max: 1
  }
  type: BYTES
  domain: "gender"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "active_member"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  bool_domain {
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "age"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "balance"
  value_count {
    min: 1
    max: 1
  }
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "churn"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  int_domain {
    name: "churn"
    min: 0
    max: 1
    is_categorical: true
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  

In [116]:
schema_importer = ImporterNode(
    instance_name='Schema_Importer',
    source_uri=schema_dir,
    artifact_type=tfx.types.standard_artifacts.Schema,
    reimport=False)

In [117]:
context.run(schema_importer)

ExecutionResult(
    component_id: ImporterNode.Schema_Importer
    execution_id: 33
    outputs:
        result: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 4
        type_id: 9
        uri: "/home/jupyter/artifact-store/schema"
        create_time_since_epoch: 1681967359479
        last_update_time_since_epoch: 1681977512330
        , artifact_type: id: 9
        name: "Schema"
        )]
        ))

In [118]:
context.show(schema_importer.outputs['result'])

,Type,Presence,Valency,Domain
Feature name,,,,
'country',STRING,required,single,'country'
'gender',STRING,required,single,'gender'
'active_member',INT,required,single,-
'age',INT,required,single,-
'balance',FLOAT,required,single,-
'churn',INT,required,single,"[0,1]"
'credit_card',INT,required,single,-
'credit_score',INT,required,single,-
'customer_id',INT,required,single,-


/home/jupyter/.local/lib/python3.7/site-packages/tensorflow_data_validation/utils/display_util.py:151: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'country',"'France', 'Germany', 'Spain'"
'gender',"'Female', 'Male'"


In [119]:
example_validator = ExampleValidator(
    instance_name="Data_Validation",    
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_importer.outputs['result'])

In [120]:
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator.Data_Validation
    execution_id: 34
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 5
        type_id: 12
        uri: "/home/jupyter/artifact-store/tfx-churn-classifier/20230420_045852/ExampleValidator.Data_Validation/anomalies/5"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator.Data_Validation"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        create_time_since_epoch: 1681967451268
        last_update_time_since_epoch: 1681967451305
        , artifact_type: id: 12
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
        ))

In [121]:
train_uri = example_validator.outputs['anomalies'].get()[0].uri
train_anomalies_filename = os.path.join(train_uri, "train/anomalies.pbtxt")
!cat $train_anomalies_filename

baseline {
  feature {
    name: "country"
    value_count {
      min: 1
      max: 1
    }
    type: BYTES
    domain: "country"
    presence {
      min_fraction: 1.0
      min_count: 1
    }
  }
  feature {
    name: "gender"
    value_count {
      min: 1
      max: 1
    }
    type: BYTES
    domain: "gender"
    presence {
      min_fraction: 1.0
      min_count: 1
    }
  }
  feature {
    name: "active_member"
    value_count {
      min: 1
      max: 1
    }
    type: INT
    bool_domain {
    }
    presence {
      min_fraction: 1.0
      min_count: 1
    }
  }
  feature {
    name: "age"
    value_count {
      min: 1
      max: 1
    }
    type: INT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
  }
  feature {
    name: "balance"
    value_count {
      min: 1
      max: 1
    }
    type: FLOAT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
  }
  feature {
    name: "churn"
    value_count {
      min: 1
      max: 1
    }
    type: INT
 

In [122]:
context.show(example_validator.outputs['output'])

In [123]:
TRANSFORM_MODULE = 'preprocessing.py'
!cat {TRANSFORM_MODULE}

import tensorflow as tf
import tensorflow_transform as tft

import features

def _fill_in_missing(x):
  """Replace missing values in a SparseTensor.
  Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
  Args:
    x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
      in the second dimension.
  Returns:
    A rank 1 tensor where missing values of `x` have been filled in.
  """
  default_value = '' if x.dtype == tf.string else 0
  return tf.squeeze(
      tf.sparse.to_dense(
          tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
          default_value),
      axis=1)

def preprocessing_fn(inputs):
  """Preprocesses Covertype Dataset."""

  outputs = {}

  # Scale numerical features.
  for key in features.NUMERIC_FEATURE_KEYS:
    outputs[features.transformed_name(key)] = tft.scale_to_z_score(
        _fill_in_missing(inputs[key]))

  # Generate vocabularies and maps categorical features.
  for key in features.CATEGORIC

In [124]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_importer.outputs['result'],
    module_file=TRANSFORM_MODULE)

In [125]:
context.run(transform)

ExecutionResult(
    component_id: Transform
    execution_id: 35
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 23
        type_id: 14
        uri: "/home/jupyter/artifact-store/tfx-churn-classifier/20230420_045852/Transform/transform_graph/14"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        create_time_since_epoch: 1681975201846
        last_update_time_since_epoch: 1681977512330
        , artifact_type: id: 14
        name: "TransformGraph"
        )]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 24
        type_id: 5
        uri: "/home/jupyter/artifact-store/tfx-churn-classifier/20230420_045852/Transform/transformed_examples/14"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        create_time_since_epoch: 1681975201847
        last_update_time_since_epoch: 1681977512330
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 25
        type_id: 15
        uri: "/home/jupyter/artifact-store/tfx-churn-classifier/20230420_045852/Transform/updated_analyzer_cache/14"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        create_time_since_epoch: 1681975201847
        last_update_time_since_epoch: 1681975216840
        , artifact_type: id: 15
        name: "TransformCache"
        )]
        ))

In [126]:
os.listdir(transform.outputs['transform_graph'].get()[0].uri)

['transform_fn', 'metadata', 'transformed_metadata']

In [127]:
os.listdir(transform.outputs['transformed_examples'].get()[0].uri)

['eval', 'train']

In [128]:
transform_uri = transform.outputs['transformed_examples'].get()[0].uri
tfrecord_filenames = [os.path.join(transform_uri,  'train', name)
                      for name in os.listdir(os.path.join(transform_uri, 'train'))]
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
for tfrecord in dataset.take(2):
  example = tf.train.Example()
  example.ParseFromString(tfrecord.numpy())
  for name, feature in example.features.feature.items():
    if feature.HasField('bytes_list'):
        value = feature.bytes_list.value
    if feature.HasField('float_list'):
        value = feature.float_list.value
    if feature.HasField('int64_list'):
        value = feature.int64_list.value
    print('{}: {}'.format(name, value))
  print('******')

gender_xf: [1]
customer_id_xf: [-0.7783196568489075]
tenure_xf: [-1.0423684120178223]
credit_score_xf: [-0.32837024331092834]
churn_xf: [1]
country_xf: [0]
age_xf: [0.30339014530181885]
products_number_xf: [-0.9159955978393555]
credit_card_xf: [0.6484262347221375]
estimated_salary_xf: [0.0157352015376091]
active_member_xf: [0.9694462418556213]
balance_xf: [-1.226130723953247]
******
churn_xf: [1]
age_xf: [0.30339014530181885]
balance_xf: [1.3369983434677124]
tenure_xf: [1.0234949588775635]
credit_score_xf: [-1.542961835861206]
gender_xf: [1]
credit_card_xf: [0.6484262347221375]
country_xf: [0]
products_number_xf: [2.5367848873138428]
estimated_salary_xf: [0.2342006415128708]
customer_id_xf: [-0.991179347038269]
active_member_xf: [-1.031516671180725]
******


In [218]:
TRAINER_MODULE_FILE = 'model.py'
# !cat {TRAINER_MODULE_FILE}

In [219]:
trainer = Trainer(
    custom_executor_spec=executor_spec.ExecutorClassSpec(trainer_executor.GenericExecutor),
    module_file=TRAINER_MODULE_FILE,
    transformed_examples=transform.outputs.transformed_examples,
    schema=schema_importer.outputs.result,
    transform_graph=transform.outputs.transform_graph,
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=5000),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=1000))

In [220]:
context.run(trainer)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
2023-04-20 10:18:20.178554: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.


   2/5000 [..............................] - ETA: 0s - loss: 0.9075 - binary_accuracy: 0.468 - ETA: 6:02 - loss: 0.8676 - binary_accuracy: 0.4453WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0036s vs `on_train_batch_end` time: 0.1435s). Check your callbacks.


2023-04-20 10:18:23.238902: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2023-04-20 10:18:23.371040: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: /home/jupyter/artifact-store/tfx-churn-classifier/20230420_045852/Trainer/model_run/66/train/plugins/profile/2023_04_20_10_18_23
2023-04-20 10:18:23.374093: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to /home/jupyter/artifact-store/tfx-churn-classifier/20230420_045852/Trainer/model_run/66/train/plugins/profile/2023_04_20_10_18_23/tensorflow-2-3-20230419-184445.trace.json.gz
2023-04-20 10:18:23.378030: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: /home/jupyter/artifact-store/tfx-churn-classifier/20230420_045852/Trainer/model_run/66/train/plugins/profile/2023_04_20_10_18_23
2023-04-20 10:18:23.378162: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data

5000/5000 [==============================] - ETA: 37s - loss: 0.8337 - binary_accuracy: 0.486 - ETA: 23s - loss: 0.7867 - binary_accuracy: 0.51 - ETA: 19s - loss: 0.7503 - binary_accuracy: 0.54 - ETA: 16s - loss: 0.7241 - binary_accuracy: 0.56 - ETA: 15s - loss: 0.6998 - binary_accuracy: 0.58 - ETA: 14s - loss: 0.6761 - binary_accuracy: 0.61 - ETA: 13s - loss: 0.6561 - binary_accuracy: 0.63 - ETA: 13s - loss: 0.6389 - binary_accuracy: 0.65 - ETA: 12s - loss: 0.6223 - binary_accuracy: 0.67 - ETA: 12s - loss: 0.6088 - binary_accuracy: 0.68 - ETA: 12s - loss: 0.5964 - binary_accuracy: 0.69 - ETA: 11s - loss: 0.5848 - binary_accuracy: 0.70 - ETA: 11s - loss: 0.5755 - binary_accuracy: 0.71 - ETA: 11s - loss: 0.5667 - binary_accuracy: 0.72 - ETA: 11s - loss: 0.5592 - binary_accuracy: 0.72 - ETA: 11s - loss: 0.5523 - binary_accuracy: 0.73 - ETA: 10s - loss: 0.5455 - binary_accuracy: 0.73 - ETA: 10s - loss: 0.5402 - binary_accuracy: 0.74 - ETA: 10s - loss: 0.5354 - binary_accuracy: 0.74 - ETA:

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Assets written to: /home/jupyter/artifact-store/tfx-churn-classifier/20230420_045852/Trainer/model/66/serving_model_dir/assets


INFO:tensorflow:Assets written to: /home/jupyter/artifact-store/tfx-churn-classifier/20230420_045852/Trainer/model/66/serving_model_dir/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 66
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 114
        type_id: 17
        uri: "/home/jupyter/artifact-store/tfx-churn-classifier/20230420_045852/Trainer/model/66"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 17
        name: "Model"
        )]
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 115
        type_id: 18
        uri: "/home/jupyter/artifact-store/tfx-churn-classifier/20230420_045852/Trainer/model_run/66"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "ModelRun"
        )]
        ))

In [221]:
tuner = Tuner(
        module_file=TRAINER_MODULE_FILE,
        examples=transform.outputs['transformed_examples'],
        transform_graph=transform.outputs['transform_graph'],
        train_args=trainer_pb2.TrainArgs(num_steps=1000),
        eval_args=trainer_pb2.EvalArgs(num_steps=500))

In [222]:
context.run(tuner)

1000/1000 [==============================] - ETA: 0s - loss: 0.7902 - binary_accuracy: 0.406 - ETA: 1s - loss: 0.7515 - binary_accuracy: 0.467 - ETA: 1s - loss: 0.7359 - binary_accuracy: 0.493 - ETA: 1s - loss: 0.7248 - binary_accuracy: 0.507 - ETA: 1s - loss: 0.7154 - binary_accuracy: 0.525 - ETA: 1s - loss: 0.7035 - binary_accuracy: 0.545 - ETA: 1s - loss: 0.6936 - binary_accuracy: 0.565 - ETA: 1s - loss: 0.6845 - binary_accuracy: 0.584 - ETA: 1s - loss: 0.6768 - binary_accuracy: 0.600 - ETA: 1s - loss: 0.6681 - binary_accuracy: 0.618 - ETA: 1s - loss: 0.6591 - binary_accuracy: 0.636 - ETA: 1s - loss: 0.6513 - binary_accuracy: 0.649 - ETA: 1s - loss: 0.6441 - binary_accuracy: 0.661 - ETA: 1s - loss: 0.6374 - binary_accuracy: 0.671 - ETA: 1s - loss: 0.6314 - binary_accuracy: 0.679 - ETA: 1s - loss: 0.6250 - binary_accuracy: 0.687 - ETA: 1s - loss: 0.6192 - binary_accuracy: 0.694 - ETA: 1s - loss: 0.6133 - binary_accuracy: 0.701 - ETA: 1s - loss: 0.6085 - binary_accuracy: 0.706 - ETA: 

1000/1000 [==============================] - ETA: 0s - loss: 0.8619 - binary_accuracy: 0.468 - ETA: 1s - loss: 0.8025 - binary_accuracy: 0.484 - ETA: 1s - loss: 0.7894 - binary_accuracy: 0.494 - ETA: 1s - loss: 0.7836 - binary_accuracy: 0.496 - ETA: 1s - loss: 0.7674 - binary_accuracy: 0.511 - ETA: 1s - loss: 0.7521 - binary_accuracy: 0.520 - ETA: 1s - loss: 0.7409 - binary_accuracy: 0.530 - ETA: 1s - loss: 0.7304 - binary_accuracy: 0.539 - ETA: 1s - loss: 0.7227 - binary_accuracy: 0.544 - ETA: 1s - loss: 0.7144 - binary_accuracy: 0.553 - ETA: 1s - loss: 0.7063 - binary_accuracy: 0.562 - ETA: 1s - loss: 0.6982 - binary_accuracy: 0.570 - ETA: 1s - loss: 0.6916 - binary_accuracy: 0.577 - ETA: 1s - loss: 0.6857 - binary_accuracy: 0.584 - ETA: 1s - loss: 0.6799 - binary_accuracy: 0.590 - ETA: 1s - loss: 0.6733 - binary_accuracy: 0.599 - ETA: 1s - loss: 0.6668 - binary_accuracy: 0.607 - ETA: 1s - loss: 0.6607 - binary_accuracy: 0.615 - ETA: 1s - loss: 0.6561 - binary_accuracy: 0.621 - ETA: 

1000/1000 [==============================] - ETA: 0s - loss: 0.7695 - binary_accuracy: 0.578 - ETA: 2s - loss: 0.8162 - binary_accuracy: 0.448 - ETA: 2s - loss: 0.7526 - binary_accuracy: 0.498 - ETA: 2s - loss: 0.7000 - binary_accuracy: 0.572 - ETA: 2s - loss: 0.6611 - binary_accuracy: 0.625 - ETA: 1s - loss: 0.6288 - binary_accuracy: 0.659 - ETA: 1s - loss: 0.6093 - binary_accuracy: 0.679 - ETA: 1s - loss: 0.5879 - binary_accuracy: 0.699 - ETA: 1s - loss: 0.5744 - binary_accuracy: 0.710 - ETA: 1s - loss: 0.5603 - binary_accuracy: 0.721 - ETA: 1s - loss: 0.5497 - binary_accuracy: 0.728 - ETA: 1s - loss: 0.5386 - binary_accuracy: 0.736 - ETA: 1s - loss: 0.5299 - binary_accuracy: 0.743 - ETA: 1s - loss: 0.5209 - binary_accuracy: 0.749 - ETA: 1s - loss: 0.5156 - binary_accuracy: 0.753 - ETA: 1s - loss: 0.5132 - binary_accuracy: 0.754 - ETA: 1s - loss: 0.5091 - binary_accuracy: 0.757 - ETA: 1s - loss: 0.5054 - binary_accuracy: 0.760 - ETA: 1s - loss: 0.5011 - binary_accuracy: 0.763 - ETA: 

1000/1000 [==============================] - ETA: 0s - loss: 0.8663 - binary_accuracy: 0.328 - ETA: 1s - loss: 0.8153 - binary_accuracy: 0.390 - ETA: 1s - loss: 0.8026 - binary_accuracy: 0.408 - ETA: 1s - loss: 0.7947 - binary_accuracy: 0.421 - ETA: 1s - loss: 0.7840 - binary_accuracy: 0.429 - ETA: 1s - loss: 0.7725 - binary_accuracy: 0.447 - ETA: 1s - loss: 0.7610 - binary_accuracy: 0.466 - ETA: 1s - loss: 0.7542 - binary_accuracy: 0.475 - ETA: 1s - loss: 0.7434 - binary_accuracy: 0.492 - ETA: 1s - loss: 0.7340 - binary_accuracy: 0.506 - ETA: 1s - loss: 0.7242 - binary_accuracy: 0.522 - ETA: 1s - loss: 0.7152 - binary_accuracy: 0.536 - ETA: 1s - loss: 0.7085 - binary_accuracy: 0.545 - ETA: 1s - loss: 0.7001 - binary_accuracy: 0.558 - ETA: 1s - loss: 0.6927 - binary_accuracy: 0.569 - ETA: 1s - loss: 0.6851 - binary_accuracy: 0.579 - ETA: 1s - loss: 0.6773 - binary_accuracy: 0.590 - ETA: 1s - loss: 0.6706 - binary_accuracy: 0.598 - ETA: 1s - loss: 0.6648 - binary_accuracy: 0.605 - ETA: 

1000/1000 [==============================] - ETA: 0s - loss: 0.6948 - binary_accuracy: 0.593 - ETA: 1s - loss: 0.7319 - binary_accuracy: 0.477 - ETA: 1s - loss: 0.7120 - binary_accuracy: 0.517 - ETA: 1s - loss: 0.6867 - binary_accuracy: 0.567 - ETA: 1s - loss: 0.6692 - binary_accuracy: 0.600 - ETA: 1s - loss: 0.6540 - binary_accuracy: 0.626 - ETA: 1s - loss: 0.6374 - binary_accuracy: 0.648 - ETA: 1s - loss: 0.6242 - binary_accuracy: 0.666 - ETA: 1s - loss: 0.6101 - binary_accuracy: 0.683 - ETA: 1s - loss: 0.5973 - binary_accuracy: 0.696 - ETA: 1s - loss: 0.5869 - binary_accuracy: 0.707 - ETA: 1s - loss: 0.5776 - binary_accuracy: 0.715 - ETA: 1s - loss: 0.5680 - binary_accuracy: 0.724 - ETA: 1s - loss: 0.5619 - binary_accuracy: 0.728 - ETA: 1s - loss: 0.5542 - binary_accuracy: 0.735 - ETA: 1s - loss: 0.5468 - binary_accuracy: 0.740 - ETA: 1s - loss: 0.5399 - binary_accuracy: 0.745 - ETA: 0s - loss: 0.5361 - binary_accuracy: 0.748 - ETA: 0s - loss: 0.5331 - binary_accuracy: 0.750 - ETA: 

...

1000/1000 [==============================] - ETA: 0s - loss: 0.7289 - binary_accuracy: 0.609 - ETA: 1s - loss: 0.5240 - binary_accuracy: 0.755 - ETA: 1s - loss: 0.4826 - binary_accuracy: 0.783 - ETA: 1s - loss: 0.4733 - binary_accuracy: 0.790 - ETA: 1s - loss: 0.4619 - binary_accuracy: 0.796 - ETA: 1s - loss: 0.4547 - binary_accuracy: 0.800 - ETA: 1s - loss: 0.4534 - binary_accuracy: 0.800 - ETA: 1s - loss: 0.4518 - binary_accuracy: 0.800 - ETA: 1s - loss: 0.4483 - binary_accuracy: 0.803 - ETA: 1s - loss: 0.4455 - binary_accuracy: 0.805 - ETA: 1s - loss: 0.4456 - binary_accuracy: 0.804 - ETA: 1s - loss: 0.4438 - binary_accuracy: 0.805 - ETA: 1s - loss: 0.4402 - binary_accuracy: 0.807 - ETA: 1s - loss: 0.4406 - binary_accuracy: 0.806 - ETA: 1s - loss: 0.4417 - binary_accuracy: 0.806 - ETA: 1s - loss: 0.4407 - binary_accuracy: 0.806 - ETA: 1s - loss: 0.4410 - binary_accuracy: 0.806 - ETA: 1s - loss: 0.4406 - binary_accuracy: 0.807 - ETA: 1s - loss: 0.4405 - binary_accuracy: 0.806 - ETA: 

1000/1000 [==============================] - ETA: 0s - loss: 0.6989 - binary_accuracy: 0.531 - ETA: 1s - loss: 0.5128 - binary_accuracy: 0.750 - ETA: 1s - loss: 0.4885 - binary_accuracy: 0.775 - ETA: 1s - loss: 0.4675 - binary_accuracy: 0.785 - ETA: 1s - loss: 0.4633 - binary_accuracy: 0.789 - ETA: 1s - loss: 0.4513 - binary_accuracy: 0.797 - ETA: 1s - loss: 0.4465 - binary_accuracy: 0.801 - ETA: 1s - loss: 0.4473 - binary_accuracy: 0.801 - ETA: 1s - loss: 0.4455 - binary_accuracy: 0.802 - ETA: 1s - loss: 0.4445 - binary_accuracy: 0.803 - ETA: 1s - loss: 0.4456 - binary_accuracy: 0.803 - ETA: 1s - loss: 0.4448 - binary_accuracy: 0.803 - ETA: 1s - loss: 0.4428 - binary_accuracy: 0.804 - ETA: 1s - loss: 0.4414 - binary_accuracy: 0.805 - ETA: 1s - loss: 0.4415 - binary_accuracy: 0.805 - ETA: 1s - loss: 0.4415 - binary_accuracy: 0.805 - ETA: 1s - loss: 0.4391 - binary_accuracy: 0.807 - ETA: 1s - loss: 0.4390 - binary_accuracy: 0.807 - ETA: 0s - loss: 0.4385 - binary_accuracy: 0.807 - ETA: 

INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


ExecutionResult(
    component_id: Tuner
    execution_id: 67
    outputs:
        best_hyperparameters: Channel(
            type_name: HyperParameters
            artifacts: [Artifact(artifact: id: 116
        type_id: 20
        uri: "/home/jupyter/artifact-store/tfx-churn-classifier/20230420_045852/Tuner/best_hyperparameters/67"
        custom_properties {
          key: "name"
          value {
            string_value: "best_hyperparameters"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Tuner"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "HyperParameters"
        )]
        ))

In [223]:
hparams_importer = ImporterNode(
    instance_name='import_hparams',
    # This can be Tuner's output file or manually edited file. The file contains
    # text format of hyperparameters (kerastuner.HyperParameters.get_config())
    source_uri=tuner.outputs.best_hyperparameters.get()[0].uri,
    artifact_type=HyperParameters)

In [224]:
context.run(hparams_importer)

ExecutionResult(
    component_id: ImporterNode.import_hparams
    execution_id: 68
    outputs:
        result: Channel(
            type_name: HyperParameters
            artifacts: [Artifact(artifact: id: 116
        type_id: 20
        uri: "/home/jupyter/artifact-store/tfx-churn-classifier/20230420_045852/Tuner/best_hyperparameters/67"
        custom_properties {
          key: "name"
          value {
            string_value: "best_hyperparameters"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Tuner"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        create_time_since_epoch: 1681985931806
        last_update_time_since_epoch: 1681985986627
        , artifact_type: id: 20
        name: "HyperParameters"
        )]
        ))

In [225]:
trainer = Trainer(
    custom_executor_spec=executor_spec.ExecutorClassSpec(trainer_executor.GenericExecutor),
    module_file=TRAINER_MODULE_FILE,
    transformed_examples=transform.outputs.transformed_examples,
    schema=schema_importer.outputs.result,
    transform_graph=transform.outputs.transform_graph,
    hyperparameters=hparams_importer.outputs.result,    
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=5000),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=1000))

In [226]:
context.run(trainer)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
2023-04-20 10:22:52.522093: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.


   1/5000 [..............................] - ETA: 0s - loss: 0.6542 - binary_accuracy: 0.6094

2023-04-20 10:22:55.414830: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.


   2/5000 [..............................] - ETA: 24:21 - loss: 0.6591 - binary_accuracy: 0.6016WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0054s vs `on_train_batch_end` time: 0.5806s). Check your callbacks.


...

5000/5000 [==============================] - ETA: 2:06 - loss: 0.6226 - binary_accuracy: 0.6775 - ETA: 1:09 - loss: 0.5962 - binary_accuracy: 0.705 - ETA: 51s - loss: 0.5675 - binary_accuracy: 0.741 - ETA: 41s - loss: 0.5477 - binary_accuracy: 0.75 - ETA: 34s - loss: 0.5307 - binary_accuracy: 0.76 - ETA: 29s - loss: 0.5161 - binary_accuracy: 0.77 - ETA: 26s - loss: 0.5063 - binary_accuracy: 0.78 - ETA: 24s - loss: 0.4981 - binary_accuracy: 0.78 - ETA: 22s - loss: 0.4911 - binary_accuracy: 0.78 - ETA: 21s - loss: 0.4869 - binary_accuracy: 0.78 - ETA: 19s - loss: 0.4836 - binary_accuracy: 0.78 - ETA: 19s - loss: 0.4784 - binary_accuracy: 0.79 - ETA: 18s - loss: 0.4747 - binary_accuracy: 0.79 - ETA: 17s - loss: 0.4716 - binary_accuracy: 0.79 - ETA: 16s - loss: 0.4698 - binary_accuracy: 0.79 - ETA: 16s - loss: 0.4693 - binary_accuracy: 0.79 - ETA: 15s - loss: 0.4666 - binary_accuracy: 0.79 - ETA: 15s - loss: 0.4650 - binary_accuracy: 0.79 - ETA: 14s - loss: 0.4639 - binary_accuracy: 0.79 -

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Assets written to: /home/jupyter/artifact-store/tfx-churn-classifier/20230420_045852/Trainer/model/69/serving_model_dir/assets


INFO:tensorflow:Assets written to: /home/jupyter/artifact-store/tfx-churn-classifier/20230420_045852/Trainer/model/69/serving_model_dir/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 69
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 117
        type_id: 17
        uri: "/home/jupyter/artifact-store/tfx-churn-classifier/20230420_045852/Trainer/model/69"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 17
        name: "Model"
        )]
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 118
        type_id: 18
        uri: "/home/jupyter/artifact-store/tfx-churn-classifier/20230420_045852/Trainer/model_run/69"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "ModelRun"
        )]
        ))

In [227]:
model_resolver = ResolverNode(
      instance_name='latest_blessed_model_resolver',
      resolver_class=latest_blessed_model_resolver.LatestBlessedModelResolver,
      model=Channel(type=Model),
      model_blessing=Channel(type=ModelBlessing))

In [228]:
context.run(model_resolver)

ExecutionResult(
    component_id: ResolverNode.latest_blessed_model_resolver
    execution_id: 70
    outputs:
        model: Channel(
            type_name: Model
            artifacts: []
        )
        model_blessing: Channel(
            type_name: ModelBlessing
            artifacts: []
        ))

In [244]:
accuracy_threshold = tfma.MetricThreshold(
                value_threshold=tfma.GenericValueThreshold(
                    lower_bound={'value': 0.5},
                    upper_bound={'value': 0.99})
                )

metrics_specs = tfma.MetricsSpec(
                   metrics = [
                       tfma.MetricConfig(class_name='BinaryAccuracy',
                           threshold=accuracy_threshold),
                       tfma.MetricConfig(class_name='ExampleCount')])

eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(label_key='churn')
    ],
    metrics_specs=[metrics_specs],
    slicing_specs=[
        tfma.SlicingSpec(),
        tfma.SlicingSpec(feature_keys=['country'])
    ]
)
eval_config

model_specs {
  label_key: "churn"
}
slicing_specs {
}
slicing_specs {
  feature_keys: "country"
}
metrics_specs {
  metrics {
    class_name: "BinaryAccuracy"
    threshold {
      value_threshold {
        lower_bound {
          value: 0.5
        }
        upper_bound {
          value: 0.99
        }
      }
    }
  }
  metrics {
    class_name: "ExampleCount"
  }
}

In [245]:
model_analyzer = Evaluator(
    examples=example_gen.outputs.examples,
    model=trainer.outputs.model,
    baseline_model=model_resolver.outputs.model,
    eval_config=eval_config
)

In [246]:
context.run(model_analyzer, enable_cache=False)

        please use "has_baseline" instead.


ExecutionResult(
    component_id: Evaluator
    execution_id: 74
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(artifact: id: 125
        type_id: 23
        uri: "/home/jupyter/artifact-store/tfx-churn-classifier/20230420_045852/Evaluator/evaluation/74"
        custom_properties {
          key: "name"
          value {
            string_value: "evaluation"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "ModelEvaluation"
        )]
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 126
        type_id: 24
        uri: "/home/jupyter/artifact-store/tfx-churn-classifier/20230420_045852/Evaluator/blessing/74"
        custom_properties {
          key: "blessed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "current_model"
          value {
            string_value: "/home/jupyter/artifact-store/tfx-churn-classifier/20230420_045852/Trainer/model/69"
          }
        }
        custom_properties {
          key: "current_model_id"
          value {
            int_value: 117
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 24
        name: "ModelBlessing"
        )]
        ))

In [247]:
model_blessing_uri = model_analyzer.outputs.blessing.get()[0].uri
!ls -l {model_blessing_uri}

total 0
-rw-r--r-- 1 jupyter jupyter 0 Apr 20 10:36 BLESSED


In [248]:
evaluation_uri = model_analyzer.outputs['evaluation'].get()[0].uri
evaluation_uri
!ls {evaluation_uri}

eval_config.json  metrics  plots  validations


In [249]:
eval_result = tfma.load_eval_result(evaluation_uri)
eval_result

EvalResult(slicing_metrics=[((), {'': {'': {'binary_accuracy': {'doubleValue': 0.8109477758407593}, 'example_count': {'doubleValue': 1973.0}}}}), ((('country', 'Spain'),), {'': {'': {'binary_accuracy': {'doubleValue': 0.826347291469574}, 'example_count': {'doubleValue': 501.0}}}}), ((('country', 'France'),), {'': {'': {'binary_accuracy': {'doubleValue': 0.8385416865348816}, 'example_count': {'doubleValue': 960.0}}}}), ((('country', 'Germany'),), {'': {'': {'binary_accuracy': {'doubleValue': 0.744140625}, 'example_count': {'doubleValue': 512.0}}}})], plots=[((), None), ((('country', 'Spain'),), None), ((('country', 'France'),), None), ((('country', 'Germany'),), None)], config=model_specs {
  label_key: "churn"
}
slicing_specs {
}
slicing_specs {
  feature_keys: "country"
}
metrics_specs {
  metrics {
    class_name: "BinaryAccuracy"
    threshold {
      value_threshold {
        lower_bound {
          value: 0.5
        }
        upper_bound {
          value: 0.99
        }
      }


In [250]:
tfma.view.render_slicing_metrics(eval_result)

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Overall', 'metrics':…

In [252]:
tfma.view.render_slicing_metrics(
    eval_result, slicing_column='country')

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'country:Spain', 'met…

In [253]:
infra_validator = InfraValidator(
    model=trainer.outputs['model'],
    examples=example_gen.outputs['examples'],
    serving_spec=infra_validator_pb2.ServingSpec(
        tensorflow_serving=infra_validator_pb2.TensorFlowServing(
            tags=['latest']),
      local_docker=infra_validator_pb2.LocalDockerConfig(),
  ),
    validation_spec=infra_validator_pb2.ValidationSpec(
        max_loading_time_seconds=60,
        num_tries=5,
    ),    
  request_spec=infra_validator_pb2.RequestSpec(
      tensorflow_serving=infra_validator_pb2.TensorFlowServingRequestSpec(),
          num_examples=5,
      )
)

In [254]:
context.run(infra_validator, enable_cache=False)

2023-04-20 10:40:29.455307: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:581] No property is defined for the Type


ExecutionResult(
    component_id: InfraValidator
    execution_id: 75
    outputs:
        blessing: Channel(
            type_name: InfraBlessing
            artifacts: [Artifact(artifact: id: 127
        type_id: 26
        uri: "/home/jupyter/artifact-store/tfx-churn-classifier/20230420_045852/InfraValidator/blessing/75"
        custom_properties {
          key: "blessed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "InfraValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 26
        name: "InfraBlessing"
        )]
        ))

In [255]:
infra_blessing_uri = infra_validator.outputs.blessing.get()[0].uri
!ls -l {infra_blessing_uri}

total 0
-rw-r--r-- 1 jupyter jupyter 0 Apr 20 10:40 INFRA_BLESSED


In [256]:
trainer.outputs['model']

Channel(
    type_name: Model
    artifacts: [Artifact(artifact: id: 117
type_id: 17
uri: "/home/jupyter/artifact-store/tfx-churn-classifier/20230420_045852/Trainer/model/69"
custom_properties {
  key: "name"
  value {
    string_value: "model"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "Trainer"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
state: LIVE
, artifact_type: id: 17
name: "Model"
)]
)

In [257]:
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=model_analyzer.outputs['blessing'],
    infra_blessing=infra_validator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=SERVING_MODEL_DIR)))
context.run(pusher)

2023-04-20 10:42:50.697833: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:581] No property is defined for the Type


ExecutionResult(
    component_id: Pusher
    execution_id: 76
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(artifact: id: 128
        type_id: 28
        uri: "/home/jupyter/artifact-store/tfx-churn-classifier/20230420_045852/Pusher/pushed_model/76"
        custom_properties {
          key: "name"
          value {
            string_value: "pushed_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Pusher"
          }
        }
        custom_properties {
          key: "pushed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "pushed_destination"
          value {
            string_value: "/home/jupyter/serving_model/1681987370"
          }
        }
        custom_properties {
          key: "pushed_version"
          value {
            string_value: "1681987370"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 28
        name: "PushedModel"
        )]
        ))

In [258]:
pusher.outputs

{'pushed_model': Channel(
    type_name: PushedModel
    artifacts: [Artifact(artifact: id: 128
type_id: 28
uri: "/home/jupyter/artifact-store/tfx-churn-classifier/20230420_045852/Pusher/pushed_model/76"
custom_properties {
  key: "name"
  value {
    string_value: "pushed_model"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "Pusher"
  }
}
custom_properties {
  key: "pushed"
  value {
    int_value: 1
  }
}
custom_properties {
  key: "pushed_destination"
  value {
    string_value: "/home/jupyter/serving_model/1681987370"
  }
}
custom_properties {
  key: "pushed_version"
  value {
    string_value: "1681987370"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
state: LIVE
, artifact_type: id: 28
name: "PushedModel"
)]
)}

In [259]:
PATH=%env PATH
%env PATH=/opt/conda/envs/tfx/bin:{PATH}

env: PATH=/opt/conda/envs/tfx/bin:/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.local/bin


In [260]:
latest_pushed_model = os.path.join(SERVING_MODEL_DIR, max(os.listdir(SERVING_MODEL_DIR)))
!saved_model_cli show --dir {latest_pushed_model} --all

2023-04-20 10:44:36.403320: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-04-20 10:44:36.403384: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.

MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['exam